In [1]:
import random
import string
import os
import collections
import multiprocessing as mp
from functools import reduce

In [2]:
def create_file(file_name, file_size):
  str_arr = []
  if not os.path.isfile(file_name):
    open(file_name, "w").close()
  while os.path.getsize(file_name) / (1024 * 1024 * file_size) < 1:
    for i in range(1024):
      str_arr.append(''.join(random.choice(string.ascii_letters) for j in range(random.randint(32, 62))) + '\n')
    chunk_of_strs = ''.join(str_arr)
    str_arr[:] = []

    with open(file_name, 'a+', encoding='utf-8') as file:
      file.write(chunk_of_strs)

In [3]:
def mapper(chunk_start, chunk_size, file_name):
    with open(file_name) as file:
        file.seek(chunk_start)
        file_content = file.read(chunk_size).splitlines()
        counter = collections.Counter(file_content)
        return counter


def reducer(counter_1, counter_2):
    return counter_1 + counter_2

In [4]:
def split_into_chunks(file_name, chunk_size=1024 ** 2):
    end_of_file = os.path.getsize(file_name)
    with open(file_name, 'rb') as file:
        chunk_end = file.tell()
        while True:
            chunk_start = chunk_end
            file.seek(chunk_size, 1)
            file.readline()
            chunk_end = file.tell()

            yield chunk_start, chunk_end - chunk_start

            if chunk_end > end_of_file:
                break

In [10]:
if __name__ == '__main__':
    FILE = "input_data.txt"
    create_file(FILE, 4)
    workers_pool = mp.Pool(8)

    task_queue = []

    for chunk_start, chunk_size in split_into_chunks(FILE):
        task_queue.append(workers_pool.apply_async(mapper, (chunk_start, chunk_size, FILE)))
    result = []
    for t in task_queue:
        result.append(t.get())
    workers_pool.close()
    common_counter = reduce(reducer, result)
    result = sorted(common_counter.items(), key=lambda pair: pair[1], reverse=True)

    print(common_counter.most_common(10))
    print(result[:10])
    print(max(result,key=lambda item:item[1]))
    print(min(result,key=lambda item:item[1]))

[('aElwFGnpYaaMTxesxTgDwzlRnjsuraMIDJvlfwEbDasiD', 1), ('kwJbKpfXkgiIDuCtyiCuMxNgowoLMNtIKlUYQLjeCmuOouIwwE', 1), ('GoleQhDsMLSnbXMPzATceSxCkWVjhUQcfBmVlhcRLWHiGNXOU', 1), ('pLizilWqCOtfoTBIIAjqLjwzJopVTTprmgWqvAMSZyBGTLCmIxt', 1), ('fMhDDujxWEzvdzyAKtrMsCramTAgSFYwZgOsTULshocvqtzHc', 1), ('UHtdDtBvgXAHYMwxTCwIsosOClvMlCclhqGztjCxYKNRQWQUMWxdAlQEOuQvIL', 1), ('PhduogRXiLTPIpkihrIadAIJhdwzlcuaKyknElmKcbYmrjycBN', 1), ('FrRaMurUgekFVqLamJkZOVSOnGUvhlLusQnHZZnHMTodVqwPrtjjpAtetEWHO', 1), ('sWOXFxGGKhgcGuHPiNoQGtBgioqHYicesSRjOeosQzuod', 1), ('QubsDFcOMDsUHPXxVmBtIjohnUFjNnoooR', 1)]
[('aElwFGnpYaaMTxesxTgDwzlRnjsuraMIDJvlfwEbDasiD', 1), ('kwJbKpfXkgiIDuCtyiCuMxNgowoLMNtIKlUYQLjeCmuOouIwwE', 1), ('GoleQhDsMLSnbXMPzATceSxCkWVjhUQcfBmVlhcRLWHiGNXOU', 1), ('pLizilWqCOtfoTBIIAjqLjwzJopVTTprmgWqvAMSZyBGTLCmIxt', 1), ('fMhDDujxWEzvdzyAKtrMsCramTAgSFYwZgOsTULshocvqtzHc', 1), ('UHtdDtBvgXAHYMwxTCwIsosOClvMlCclhqGztjCxYKNRQWQUMWxdAlQEOuQvIL', 1), ('PhduogRXiLTPIpkihrIadAIJhdwzlcuaKyknElmKcbYmrjycBN